<a href="https://colab.research.google.com/github/Sanyam8055/EIP-4.0-/blob/master/FinalAssignment5/CustomResnet_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preprocessing

In [0]:
from google.colab import drive
drive.mount('/content/drive')
#!unzip -q "/content/drive/My Drive/Colab Notebooks/hvc_data.zip"
# look for `hvc_annotations.csv` file and `resized` dir
%ls
%tensorflow_version 1.x
!pip install bokeh
import cv2
import json
from keras.callbacks import ModelCheckpoint
import numpy as np
import pandas as pd

from functools import partial
from pathlib import Path 
from tqdm import tqdm

from google.colab.patches import cv2_imshow

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder


from keras.applications import VGG16,InceptionV3, ResNet50V2
from keras.layers.core import Dropout
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.layers import Input
from keras.models import Model
from keras.optimizers import Adam,SGD,RMSprop
from keras.preprocessing.image import ImageDataGenerator
# load annotations
df = pd.read_csv("hvc_annotations.csv")
del df["filename"] # remove unwanted column

df.head()
# one hot encoding of labels

one_hot_df = pd.concat([
    df[["image_path"]],
    pd.get_dummies(df.gender, prefix="gender"),
    pd.get_dummies(df.imagequality, prefix="imagequality"),
    pd.get_dummies(df.age, prefix="age"),
    pd.get_dummies(df.weight, prefix="weight"),
    pd.get_dummies(df.carryingbag, prefix="carryingbag"),
    pd.get_dummies(df.footwear, prefix="footwear"),
    pd.get_dummies(df.emotion, prefix="emotion"),
    pd.get_dummies(df.bodypose, prefix="bodypose"),
], axis = 1)

one_hot_df.head().T
import keras
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
# Label columns per attribute
_gender_cols_ = [col for col in one_hot_df.columns if col.startswith("gender")]
_imagequality_cols_ = [col for col in one_hot_df.columns if col.startswith("imagequality")]
_age_cols_ = [col for col in one_hot_df.columns if col.startswith("age")]
_weight_cols_ = [col for col in one_hot_df.columns if col.startswith("weight")]
_carryingbag_cols_ = [col for col in one_hot_df.columns if col.startswith("carryingbag")]
_footwear_cols_ = [col for col in one_hot_df.columns if col.startswith("footwear")]
_emotion_cols_ = [col for col in one_hot_df.columns if col.startswith("emotion")]
_bodypose_cols_ = [col for col in one_hot_df.columns if col.startswith("bodypose")]

class PersonDataGenerator(keras.utils.Sequence):
    """Ground truth data generator"""

    
    def __init__(self, df, batch_size=32, shuffle=True, augmentation=True):
        self.df = df
        self.batch_size=batch_size
        self.shuffle = shuffle
        self.on_epoch_end()
        self.augmentation = augmentation
    def __len__(self):
        return int(np.floor(self.df.shape[0] / self.batch_size))

    def __getitem__(self, index):
        """fetch batched images and targets"""
        batch_slice = slice(index * self.batch_size, (index + 1) * self.batch_size)
        items = self.df.iloc[batch_slice]
        image = np.stack([cv2.imread(item["image_path"]) for _, item in items.iterrows()])
        target = {
            "gender_output": items[_gender_cols_].values,
            "image_quality_output": items[_imagequality_cols_].values,
            "age_output": items[_age_cols_].values,
            "weight_output": items[_weight_cols_].values,
            "bag_output": items[_carryingbag_cols_].values,
            "pose_output": items[_bodypose_cols_].values,
            "footwear_output": items[_footwear_cols_].values,
            "emotion_output": items[_emotion_cols_].values,
        }
        return image, target

    def on_epoch_end(self):
        """Updates indexes after each epoch"""
        if self.shuffle == True:
            self.df = self.df.sample(frac=1).reset_index(drop=True)
from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(one_hot_df, test_size=0.15, random_state=1)
train_df.shape, val_df.shape
train_df.head()
# create train and validation data generators
train_gen = PersonDataGenerator(train_df, batch_size=128)
valid_gen = PersonDataGenerator(val_df, batch_size=128, shuffle=True)
# get number of output units from data
images, targets = next(iter(train_gen))
num_units = { k.split("_output")[0]:v.shape[1] for k, v in targets.items()}
num_units
from keras.layers import Input
from keras.models import Model
from keras import optimizers
from keras import initializers
from keras import layers
from keras.callbacks import *
filepath="/content/drive/My Drive/Colab Notebooks/check.ckpt"
checkpoint = ModelCheckpoint(filepath,monitor='val_loss', verbose=1,save_weights_only=False, save_best_only=True)  
 

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
drive/  hvc_annotations.csv  resized/  sample_data/


# MODEL 

In [0]:
def residual_block(y, nb_channels, kernel_size, init):

    shortcut = y

    y = layers.Conv2D(nb_channels, kernel_size=kernel_size, strides=(1, 1), padding='same', kernel_initializer=init)(y)
    y = layers.BatchNormalization()(y)
    y = layers.ReLU()(y)

    y = layers.Conv2D(nb_channels, kernel_size=kernel_size, strides=(1, 1), padding='same', kernel_initializer=init)(y)
    y = layers.BatchNormalization()(y)
    
    y = layers.add([shortcut, y])
    y = layers.BatchNormalization()(y)
    y = layers.ReLU()(y)
    

    return y
def resnet_7():
    init = initializers.glorot_normal(seed=0)
    img_height = 224
    img_width = 224
    channel = 3
    input = Input(shape=(img_height, img_width, channel,))
    x = layers.Conv2D(32, kernel_size=(3, 3), strides=(2, 2), padding='same', kernel_initializer=init, input_shape=(224,224,3))(input)
    x = layers.Conv2D(32, kernel_size=(3, 3), strides=(1,1), padding='same', kernel_initializer=init)(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D(pool_size=2, strides = (2,2))(x)
    
    x = layers.Conv2D(64, kernel_size=(3, 3),strides=(1,1), padding='same', kernel_initializer=init)(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = residual_block(x, 64, (3, 3), init)
    x = layers.Conv2D(64, kernel_size=(3,3),strides=(1,1), padding='same', kernel_initializer=init)(x)
    x = layers.MaxPooling2D(pool_size=2, strides = (2,2))(x)
    x = layers.Conv2D(128, kernel_size=(3, 3),strides=(1,1), padding='same', kernel_initializer=init)(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = residual_block(x, 128, (3, 3), init)
    x = layers.Conv2D(128, kernel_size=(3,3),strides=(1,1), padding='same', kernel_initializer=init)(x)
    x = layers.MaxPooling2D(pool_size=2, strides = (2,2))(x)


    
    x = layers.Conv2D(256, kernel_size=(3, 3),strides=(1,1), padding='same', kernel_initializer=init)(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = residual_block(x, 256, (3, 3), init)
    x = layers.Conv2D(256, kernel_size=(3,3),strides=(1,1), padding='same', kernel_initializer=init)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(256, kernel_size=(3,3), padding='same', kernel_initializer=init)(x)
    x = layers.Conv2D(256, kernel_size=(3,3), strides=1, padding='same', kernel_initializer=init)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(512, kernel_size=(3,3),padding='same',activation = 'relu')(x)
    x = layers.MaxPooling2D(pool_size=2, strides = (2,2))(x)
    x = residual_block(x, 512, (3, 3), init)
    x = layers.SeparableConv2D(512, kernel_size=(3,3),strides=1, activation = 'softmax')(x)
    x = layers.Flatten()(x)
    
    model = Model(inputs=input, outputs=x)
    return model

model = resnet_7()
  
backbone = resnet_7()


neck = backbone.output



def build_tower(in_layer):
    neck = Dropout(0.2)(in_layer)
    neck = Dense(64, activation="relu")(neck)
    neck = Dropout(0.3)(in_layer)
    neck = Dense(64, activation="relu")(neck)
    return neck


def build_head(name, in_layer):
    return Dense(
        num_units[name], activation="softmax", name=f"{name}_output"
    )(in_layer)

# heads
gender = build_head("gender", build_tower(neck))
image_quality = build_head("image_quality", build_tower(neck))
age = build_head("age", build_tower(neck))
weight = build_head("weight", build_tower(neck))
bag = build_head("bag", build_tower(neck))
footwear = build_head("footwear", build_tower(neck))
emotion = build_head("emotion", build_tower(neck))
pose = build_head("pose", build_tower(neck))


model = Model(
    inputs=backbone.input, 
    outputs=[gender, image_quality, age, weight, bag, footwear, pose, emotion]
)

from keras.callbacks import *



# model.fit(X_train, y_train, validation_data=(X_valid, y_valid), batch_size=32, epochs=10)
model.summary()















Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv2d_20 (Conv2D)              (None, 112, 112, 32) 896         input_2[0][0]                    
__________________________________________________________________________________________________
conv2d_21 (Conv2D)              (None, 112, 112, 32) 9248        conv2d_20[0][0]                  
__________________________________________________________________________________________________
batch_normalization_19 (BatchNo (None, 112, 112, 32) 128       

Try with decresing kernel size 

# LR FINDER AND CYCLIC LR

In [0]:

from keras.callbacks import *
def find_lr(model, start_lr, end_lr):
    finder = LRFinder(start_lr, end_lr, len(train_gen))
    weights = model.get_weights()    
    try:
        history = model.fit_generator(
            generator=train_gen,
            validation_data=valid_gen,
            epochs=1,
            verbose=1,
            callbacks=[finder],
        )   
    finally:    
        model.set_weights(weights)    
    return finder
    
class LRFinder(Callback):    
    def __init__(self, start_lr=1e-5, end_lr=10, step_size=None, beta=.98):
        super().__init__()
        
        self.start_lr = start_lr
        self.end_lr = end_lr
        self.step_size = step_size
        self.beta = beta
        self.lr_mult = (end_lr/start_lr)**(1/step_size)
        
    def on_train_begin(self, logs=None):
        self.best_loss = 1e9
        self.avg_loss = 0
        self.losses, self.smoothed_losses, self.lrs, self.iterations = [], [], [], []
        self.iteration = 0
        logs = logs or {}
        K.set_value(self.model.optimizer.lr, self.start_lr)
        
    def on_batch_end(self, epoch, logs=None):
        logs = logs or {}
        loss = logs.get('loss')
        self.iteration += 1
        
        self.avg_loss = self.beta * self.avg_loss + (1 - self.beta) * loss
        smoothed_loss = self.avg_loss / (1 - self.beta**self.iteration)
        
        # Check if the loss is not exploding
        if self.iteration>1 and smoothed_loss > self.best_loss * 4:
            self.model.stop_training = True
            return

        if smoothed_loss < self.best_loss or self.iteration==1:
            self.best_loss = smoothed_loss
        
        lr = self.start_lr * (self.lr_mult**self.iteration)
        
        self.losses.append(loss)
        self.smoothed_losses.append(smoothed_loss)
        self.lrs.append(lr)
        self.iterations.append(self.iteration)        
        
        K.set_value(self.model.optimizer.lr, lr)  

    def plot(self, lskip=10, rskip=10):
        lrs = self.lrs[lskip:-rskip]
        losses = self.smoothed_losses[lskip:-rskip]

        output_notebook()
        p = figure(title='Learning Rate Finder', x_axis_label='LR', y_axis_label='Loss')
        p.line(lrs, losses)
        show(p)
        
        best_idxs = np.argpartition(losses, 15)[:15]
        best_lrs = np.take(lrs, best_idxs)
        print(f"Best LRs: {best_lrs}")    
#Cyclic Learning Rate
class CyclicLR(Callback):
  

    def __init__(self, base_lr=0.00000005, max_lr=0.009, step_size=2000., mode='triangular',
                 gamma=1., scale_fn=None, scale_mode='cycle'):
        super(CyclicLR, self).__init__()

        self.base_lr = base_lr
        self.max_lr = max_lr
        self.step_size = step_size
        self.mode = mode
        self.gamma = gamma
        if scale_fn == None:
            if self.mode == 'triangular':
                self.scale_fn = lambda x: 1.
                self.scale_mode = 'cycle'
            elif self.mode == 'triangular2':
                self.scale_fn = lambda x: 1/(2.**(x-1))
                self.scale_mode = 'cycle'
            elif self.mode == 'exp_range':
                self.scale_fn = lambda x: gamma**(x)
                self.scale_mode = 'iterations'
        else:
            self.scale_fn = scale_fn
            self.scale_mode = scale_mode
        self.clr_iterations = 0.
        self.trn_iterations = 0.
        self.history = {}

        self._reset()

    def _reset(self, new_base_lr=None, new_max_lr=None,
               new_step_size=None):
        """Resets cycle iterations.
        Optional boundary/step size adjustment.
        """
        if new_base_lr != None:
            self.base_lr = new_base_lr
        if new_max_lr != None:
            self.max_lr = new_max_lr
        if new_step_size != None:
            self.step_size = new_step_size
        self.clr_iterations = 0.
        
    def clr(self):
        cycle = np.floor(1+self.clr_iterations/(2*self.step_size))
        x = np.abs(self.clr_iterations/self.step_size - 2*cycle + 1)
        if self.scale_mode == 'cycle':
            return self.base_lr + (self.max_lr-self.base_lr)*np.maximum(0, (1-x))*self.scale_fn(cycle)
        else:
            return self.base_lr + (self.max_lr-self.base_lr)*np.maximum(0, (1-x))*self.scale_fn(self.clr_iterations)
        
    def on_train_begin(self, logs={}):
        logs = logs or {}

        if self.clr_iterations == 0:
            K.set_value(self.model.optimizer.lr, self.base_lr)
        else:
            K.set_value(self.model.optimizer.lr, self.clr())        
            
    def on_batch_end(self, epoch, logs=None):
        
        logs = logs or {}
        self.trn_iterations += 1
        self.clr_iterations += 1

        self.history.setdefault('lr', []).append(K.get_value(self.model.optimizer.lr))
        self.history.setdefault('iterations', []).append(self.trn_iterations)

        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)
        
        K.set_value(self.model.optimizer.lr, self.clr())


# Optimizer and LR finder



In [0]:
opt = RMSprop()
model.compile(
    optimizer=opt,
    loss="categorical_crossentropy", 
      
    metrics=["accuracy"]
)
finder = find_lr(model, 1e-4, 1e-2)
finder.plot()


Epoch 1/1
90/90 [==============================] - 48s 529ms/step - loss: 8.2206 - gender_output_loss: 0.6879 - image_quality_output_loss: 1.0033 - age_output_loss: 1.4739 - weight_output_loss: 1.0778 - bag_output_loss: 0.9556 - footwear_output_loss: 1.0373 - pose_output_loss: 0.9667 - emotion_output_loss: 1.0182 - gender_output_acc: 0.5552 - image_quality_output_acc: 0.5523 - age_output_acc: 0.3990 - weight_output_acc: 0.6282 - bag_output_acc: 0.5600 - footwear_output_acc: 0.4726 - pose_output_acc: 0.6138 - emotion_output_acc: 0.7112 - val_loss: 8.4810 - val_gender_output_loss: 0.6880 - val_image_quality_output_loss: 1.0466 - val_age_output_loss: 1.4848 - val_weight_output_loss: 1.0564 - val_bag_output_loss: 0.9429 - val_footwear_output_loss: 1.0807 - val_pose_output_loss: 1.1100 - val_emotion_output_loss: 1.0717 - val_gender_output_acc: 0.5656 - val_image_quality_output_acc: 0.5594 - val_age_output_acc: 0.3688 - val_weight_output_acc: 0.6432 - val_bag_output_acc: 0.5443 - val_footwea

Best LRs: [0.00441006 0.00599484 0.00569581 0.0054117  0.00514175 0.00488527
 0.00464159 0.00419008 0.00398107 0.00378249 0.00359381 0.00341455
 0.00324423 0.00292864 0.0030824 ]


AUGMENTATION AND FIT GENERATOR

In [0]:
	# add image augmentation
train_gen = PersonDataGenerator(
    train_df, 
    augmentation=ImageDataGenerator(
        zca_whitening=True,
        horizontal_flip=True,
        vertical_flip=False,
    )
)
clr_triangular = CyclicLR(1.8e-4,1e-3,mode='triangular2')
history=model.fit_generator(  
    steps_per_epoch=20000 // 32,  validation_steps=2000 // 64,
    generator=train_gen,
    validation_data=valid_gen,
    use_multiprocessing=True,
    workers=8, 
    epochs=25,
    verbose=1,callbacks=[checkpoint,clr_triangular]
)

/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/image_data_generator.py:336: UserWarning: This ImageDataGenerator specifies `zca_whitening`, which overrides setting of `featurewise_center`.
  warnings.warn('This ImageDataGenerator specifies '


Epoch 1/25
  3/625 [..............................] - ETA: 15:22 - loss: 9.4109 - gender_output_loss: 0.6951 - image_quality_output_loss: 1.0969 - age_output_loss: 1.6043 - weight_output_loss: 1.3667 - bag_output_loss: 1.0913 - footwear_output_loss: 1.0925 - pose_output_loss: 1.0931 - emotion_output_loss: 1.3710 - gender_output_acc: 0.4688 - image_quality_output_acc: 0.4688 - age_output_acc: 0.3438 - weight_output_acc: 0.4688 - bag_output_acc: 0.5208 - footwear_output_acc: 0.4375 - pose_output_acc: 0.4271 - emotion_output_acc: 0.5208

/usr/local/lib/python3.6/dist-packages/keras/callbacks.py:95: RuntimeWarning: Method (on_train_batch_end) is slow compared to the batch update (0.263415). Check your callbacks.
  % (hook_name, delta_t_median), RuntimeWarning)


625/625 [==============================] - 103s 164ms/step - loss: 7.7580 - gender_output_loss: 0.6758 - image_quality_output_loss: 0.9830 - age_output_loss: 1.4287 - weight_output_loss: 0.9953 - bag_output_loss: 0.9132 - footwear_output_loss: 0.9235 - pose_output_loss: 0.9262 - emotion_output_loss: 0.9122 - gender_output_acc: 0.5764 - image_quality_output_acc: 0.5522 - age_output_acc: 0.4046 - weight_output_acc: 0.6351 - bag_output_acc: 0.5662 - footwear_output_acc: 0.5688 - pose_output_acc: 0.6159 - emotion_output_acc: 0.7179 - val_loss: 7.7799 - val_gender_output_loss: 0.6338 - val_image_quality_output_loss: 0.9765 - val_age_output_loss: 1.4383 - val_weight_output_loss: 0.9772 - val_bag_output_loss: 0.9375 - val_footwear_output_loss: 0.9363 - val_pose_output_loss: 0.9131 - val_emotion_output_loss: 0.9672 - val_gender_output_acc: 0.6300 - val_image_quality_output_acc: 0.5585 - val_age_output_acc: 0.3662 - val_weight_output_acc: 0.6406 - val_bag_output_acc: 0.5144 - val_footwear_outpu

In [0]:
plot_stats(history.history, clr_triangular.history)

In [0]:
from keras.models import load_model
model=load_model('/content/drive/My Drive/Colab Notebooks/check.ckpt')
finder = find_lr(model, 1e-7, 1e-3)
finder.plot()

Epoch 1/1
360/360 [==============================] - 55s 153ms/step - loss: 5.8320 - gender_output_loss: 0.2160 - image_quality_output_loss: 0.8718 - age_output_loss: 1.2995 - weight_output_loss: 0.8839 - bag_output_loss: 0.7421 - footwear_output_loss: 0.6571 - pose_output_loss: 0.3363 - emotion_output_loss: 0.8252 - gender_output_acc: 0.9179 - image_quality_output_acc: 0.5897 - age_output_acc: 0.4367 - weight_output_acc: 0.6478 - bag_output_acc: 0.6845 - footwear_output_acc: 0.7201 - pose_output_acc: 0.8776 - emotion_output_acc: 0.7161 - val_loss: 7.5416 - val_gender_output_loss: 0.7014 - val_image_quality_output_loss: 1.0010 - val_age_output_loss: 1.4594 - val_weight_output_loss: 0.9650 - val_bag_output_loss: 1.0066 - val_footwear_output_loss: 0.8209 - val_pose_output_loss: 0.6358 - val_emotion_output_loss: 0.9515 - val_gender_output_acc: 0.7484 - val_image_quality_output_acc: 0.5391 - val_age_output_acc: 0.3302 - val_weight_output_acc: 0.6427 - val_bag_output_acc: 0.5854 - val_footw

Best LRs: [0.00059948 0.00077426 0.00075471 0.00055519 0.00071706 0.00069895
 0.00068129 0.00066408 0.00064731 0.00063096 0.00061502 0.00058434
 0.00073564 0.00056958 0.00048853]


In [0]:
	# add image augmentation
train_gen = PersonDataGenerator(
    train_df, 
    augmentation=ImageDataGenerator(
        zca_whitening=True,
        horizontal_flip=True,
        vertical_flip=False,
    )
)
clr_triangular = CyclicLR(1e-6,1e-5,mode='triangular2')
history=model.fit_generator(  
    steps_per_epoch=20000 // 32,  validation_steps=2000 // 64,
    generator=train_gen,
    validation_data=valid_gen,
    use_multiprocessing=True,
    workers=8, 
    epochs=25,
    verbose=1,callbacks=[checkpoint,clr_triangular]
)


/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/image_data_generator.py:336: UserWarning: This ImageDataGenerator specifies `zca_whitening`, which overrides setting of `featurewise_center`.
  warnings.warn('This ImageDataGenerator specifies '


Epoch 1/25
  2/625 [..............................] - ETA: 19:21 - loss: 5.5312 - gender_output_loss: 0.2068 - image_quality_output_loss: 0.8833 - age_output_loss: 1.2440 - weight_output_loss: 0.7469 - bag_output_loss: 0.8298 - footwear_output_loss: 0.6058 - pose_output_loss: 0.2863 - emotion_output_loss: 0.7283 - gender_output_acc: 0.9219 - image_quality_output_acc: 0.5938 - age_output_acc: 0.4375 - weight_output_acc: 0.7188 - bag_output_acc: 0.6250 - footwear_output_acc: 0.7344 - pose_output_acc: 0.8750 - emotion_output_acc: 0.7656

/usr/local/lib/python3.6/dist-packages/keras/callbacks.py:95: RuntimeWarning: Method (on_train_batch_end) is slow compared to the batch update (1.276692). Check your callbacks.
  % (hook_name, delta_t_median), RuntimeWarning)
/usr/local/lib/python3.6/dist-packages/keras/callbacks.py:95: RuntimeWarning: Method (on_train_batch_end) is slow compared to the batch update (0.641567). Check your callbacks.
  % (hook_name, delta_t_median), RuntimeWarning)


625/625 [==============================] - 101s 162ms/step - loss: 5.8254 - gender_output_loss: 0.2103 - image_quality_output_loss: 0.8717 - age_output_loss: 1.2990 - weight_output_loss: 0.8827 - bag_output_loss: 0.7449 - footwear_output_loss: 0.6547 - pose_output_loss: 0.3340 - emotion_output_loss: 0.8283 - gender_output_acc: 0.9224 - image_quality_output_acc: 0.5931 - age_output_acc: 0.4371 - weight_output_acc: 0.6465 - bag_output_acc: 0.6847 - footwear_output_acc: 0.7227 - pose_output_acc: 0.8778 - emotion_output_acc: 0.7144 - val_loss: 6.6118 - val_gender_output_loss: 0.3532 - val_image_quality_output_loss: 0.9115 - val_age_output_loss: 1.3859 - val_weight_output_loss: 0.9299 - val_bag_output_loss: 0.8379 - val_footwear_output_loss: 0.7715 - val_pose_output_loss: 0.5169 - val_emotion_output_loss: 0.9049 - val_gender_output_acc: 0.8571 - val_image_quality_output_acc: 0.5731 - val_age_output_acc: 0.3934 - val_weight_output_acc: 0.6575 - val_bag_output_acc: 0.6275 - val_footwear_outpu